In [ ]:
try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle
    
from smr import File
import numpy as np
from matplotlib import pyplot as plt
from kaveh.behavioral import oculomotor
from kaveh import plots
import os
import scipy.signal


In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
# read the files, detect target jumps, and save the plots of the target position and target jump vectors to a png image file

plt.ioff()
source_path = '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2010_Adapt/'
for root, dirnames, filenames in os.walk(source_path):
    for filename in filenames:
        if filename.endswith('smr'):
            f_name = os.path.join(root,filename)
            print(f_name)
            smr_content = File(f_name)
            smr_content.read_channels()
            ht = smr_content.get_channel(3)
            vt = smr_content.get_channel(4)
            pos_2d = np.column_stack((vt.data, ht.data))
            pos_norm = np.linalg.norm(pos_2d,  axis = -1)
            pos_diff = np.abs(np.diff(pos_norm))
            target_jump_indices = scipy.signal.find_peaks(pos_diff, prominence=200)[0]

            # remove detected target jumps that are sequential (less than 5 samples apart)
            to_delete = []
            for i, tji in enumerate(target_jump_indices[1:]):
                if tji - target_jump_indices[i] < 5:
                    to_delete = to_delete + [i+1]
            mask = np.ones(target_jump_indices.shape, dtype=bool)
            mask[to_delete] = False
            target_jump_indices = target_jump_indices[mask]
            
            jump_vecs_h = []
            for tji in target_jump_indices:
                jump_vecs_h = jump_vecs_h + [ht.data[tji + 5] - ht.data[tji - 5]]
            jump_vecs_h = np.array(jump_vecs_h)    

            jump_vecs_v = []
            for tji in target_jump_indices:
                jump_vecs_v = jump_vecs_v + [vt.data[tji + 5] - vt.data[tji - 5]]
            jump_vecs_v = np.array(jump_vecs_v)
            
            jump_vecs = np.column_stack((jump_vecs_h, jump_vecs_v))
            
            f, (ax1, ax2) = plt.subplots(1, 2, figsize = (20,10))
            ax1.set_ylim((-10000, 10000))
            ax1.set_xlim((-10000, 10000))
            ax1.plot(jump_vecs_h, jump_vecs_v, '.b', alpha = 0.07)
            ax1.set_title('jump vectors')
            ax2.set_ylim((-10000, 10000))
            ax2.set_xlim((-10000, 10000))
            ax2.plot(ht.data, vt.data, '.', alpha=0.1)
            ax2.set_title('target position')
            plt.savefig(os.path.join('../data/jump_vector_plots/', filename + '.png'))
            plt.close(fig=f)
        


In [ ]:
# Read images sequentially and show them for 7 seconds

# %matplotlib inline
# import time
# # import pylab as pl
# from IPython import display
# plt.figure(figsize=(20,10))
# time.sleep(5.0)

# source_path = '../data/jump_vector_plots/'
# for root, dirnames, filenames in os.walk(source_path):
#     for filename in filenames:
#         if filename.endswith('png'):
#             f_name = os.path.join(root,filename)
#             img = plt.imread(f_name)
#             plt.imshow(img)
#             plt.title(f_name)
#             display.clear_output(wait=True)
#             display.display(plt.gcf())
#             time.sleep(7.0)
#             plt.cla()

In [ ]:
source_path = '../data/jump_vector_plots/'
for root, dirnames, filenames in os.walk(source_path):
    for filename in filenames:
        if filename.endswith('png'):
            f_name = os.path.join(root,filename)
            print(f_name)